In [1]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import corner
import json
# import cabinetry

import pytensor
from pytensor import tensor as pt
from pytensor.graph.basic import Apply
from pytensor.graph import Apply, Op

import jax
from jax import grad, jit, vmap, value_and_grad, random
import jax.numpy as jnp

import pyhf
import pyhf
from pyhf.contrib.viz import brazil
# pyhf.set_backend('jax')

import pymc as pm
import arviz as az

from Bayesian_pyhf import infer
from Bayesian_pyhf import plotting

%load_ext autoreload
%autoreload 

In [2]:
blue = '#7CA1CC' # '#A8B6CC'
rosa =  '#E57A77'

In [3]:
with open(f'ttbar_ljets_xsec_inclusive_pruned.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
workspace

model = workspace.model()

data = workspace.data(model, include_auxdata=False)

unconstr_priors = {
    'ttbarXsec': {'type': 'HalfNormal_Unconstrained', 'sigma': [0.4]}
}

act_aux_data = workspace.data(model)
n_draws = 20_000

In [4]:
n_draws = 100

with infer.model(model, unconstr_priors, data):
    # step = pm.Metropolis()
    # post_data = pm.sample(draws=n_draws, chains=1) #, step=step)
    # post_pred = pm.sample_posterior_predictive(post_data)
    prior_pred = pm.sample_prior_predictive(n_draws)

Sampling: [Expected_Data, JET_BJES_Response, JET_EffectiveNP_Detector1, JET_EffectiveNP_Mixed1, JET_EffectiveNP_Mixed2, JET_EffectiveNP_Modelling1, JET_EffectiveNP_Modelling2, JET_EffectiveNP_Modelling3, JET_EffectiveNP_Statistical1, JET_EffectiveNP_Statistical2, JET_EtaIntercalibration_Modelling, JET_EtaIntercalibration_TotalStat, JET_Flavor_Composition, JET_Flavor_Response, JET_JER_DataVsMC, JET_JER_NP1, JET_JER_NP2, JET_JER_NP3, JET_JER_NP4, JET_JER_NP5, JET_JER_NP6, JET_JER_NP7, JET_Pileup_OffsetMu, JET_Pileup_OffsetNPV, JET_Pileup_RhoTopology, Lumi, MET_SoftTrk_ResoPara, MET_SoftTrk_ResoPerp, MET_SoftTrk_Scale, MultijetNorm_SR1_ejets, MultijetNorm_SR1_mujets, MultijetNorm_SR2_ejets, MultijetNorm_SR2_mujets, MultijetNorm_SR3_ejets, MultijetNorm_SR3_mujets, MultijetShape_SR1_mujets, MultijetShape_SR2_ejets, MultijetShape_SR2_mujets, MultijetShape_SR3_ejets, MultijetShape_SR3_mujets, OtherNorm, STop_DR_DS_Model, STop_FSR_Model, STop_ISR_Model_Var3c, STop_ISR_Model_muF, STop_ISR_Model

In [5]:
n_draws = 100

with infer.model(model, unconstr_priors, data):
    post_data = pm.sample(draws=100, tune=10)

Only 100 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [JET_JER_DataVsMC]
>Metropolis: [JET_JER_NP1]
>Metropolis: [JET_JER_NP2]
>Metropolis: [JET_JER_NP3]
>Metropolis: [JET_JER_NP4]
>Metropolis: [JET_JER_NP7]
>Metropolis: [weight_bTagSF_MV2c10_Continuous_eigenvars_C_0]
>Metropolis: [weight_bTagSF_MV2c10_Continuous_eigenvars_C_1]
>Metropolis: [weight_bTagSF_MV2c10_Continuous_eigenvars_Light_0]
>Metropolis: [weight_bTagSF_MV2c10_Continuous_eigenvars_Light_1]
>Metropolis: [weight_jvt]
>Metropolis: [weight_pileup]
>Metropolis: [MultijetNorm_SR1_ejets]
>Metropolis: [MultijetNorm_SR1_mujets]
>Metropolis: [MultijetShape_SR1_mujets]
>Metropolis: [JET_EtaIntercalibration_Modelling]
>Metropolis: [JET_Flavor_Response]
>Metropolis: [JET_Pileup_RhoTopology]
>Metropolis: [ttbar_FSR_Model_SR1]
>Metropolis: [ttbar_ISR_Model_Var3c_SR1]
>Metropolis: [ttbar_ISR_Model_hdamp]
>Metropolis: [ttbar_ISR_Model_muF_SR1]
>Metropolis: [ttbar_ISR_Model_muR_SR1]
>Metropolis: 

/home/iwsatlas1/mhorstm/.local/lib/python3.11/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/iwsatlas1/mhorstm/.local/lib/python3.11/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/iwsatlas1/mhorstm/.local/lib/python3.11/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/home/iwsatlas1/mhorstm/.local/lib/python3.11/site-packages/pymc/step_methods/metropolis.py:285: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
Sampling 4 chains for 10 tune and 100 draw iterations (40 + 400 draws total) took 47 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08